In [1]:
# для автозагрузки изменений в модулях без необходимости перезагрузки kelner
%load_ext autoreload
%autoreload 2

# необходимо для корректного импорта своих модулей в JupyterLab
import sys
current_dir = sys.path[0]
project_dir = 'SF_DS_Pro'
last_position = current_dir.find(project_dir) + len(project_dir)
project_path = current_dir[:last_position]
sys.path.append(project_path)

import Handlers as hd

In [2]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации
%matplotlib inline
plt.style.use('seaborn') #установка стиля matplotlib

In [3]:
from sklearn.model_selection import train_test_split
from sklearn import ensemble # ансамбли
from sklearn import metrics # метрики

### Задание 6.5
Вам необходимо будет предсказать качество вина по его характеристикам.

In [4]:
wine_data = pd.read_csv('data/winequality-red.zip', sep=';')
wine_data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [5]:
wine_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [6]:
wine_data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [7]:
wine_data['free sulfur dioxide'].value_counts()

6.0     138
5.0     104
10.0     79
15.0     78
12.0     75
7.0      71
9.0      62
16.0     61
17.0     60
11.0     59
13.0     57
8.0      56
14.0     50
3.0      49
18.0     46
4.0      41
21.0     41
19.0     39
24.0     34
23.0     32
26.0     32
20.0     30
27.0     29
25.0     24
28.0     23
29.0     23
32.0     22
22.0     22
31.0     20
34.0     18
30.0     16
35.0     15
33.0     11
36.0     11
38.0      9
41.0      7
40.0      6
39.0      5
48.0      4
51.0      4
42.0      3
43.0      3
52.0      3
37.0      3
1.0       3
45.0      3
55.0      2
37.5      2
50.0      2
68.0      2
5.5       1
53.0      1
2.0       1
54.0      1
47.0      1
57.0      1
72.0      1
66.0      1
40.5      1
46.0      1
Name: free sulfur dioxide, dtype: int64

In [8]:
wine_data['total sulfur dioxide'].value_counts()

28.0     43
24.0     36
18.0     35
15.0     35
23.0     34
         ..
142.0     1
126.0     1
139.0     1
140.0     1
160.0     1
Name: total sulfur dioxide, Length: 144, dtype: int64

Видно что у данных признаков избыточная точность.

Привожу их к целочисленному типу.

Но к сожалению на метриках это не сказалось.

In [9]:
wine_data['free sulfur dioxide'] = wine_data['free sulfur dioxide'].round().astype('int')
wine_data['total sulfur dioxide'] = wine_data['total sulfur dioxide'].round().astype('int')

Дубликаты

In [10]:
wine_data[wine_data.duplicated()]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4,7.4,0.700,0.00,1.90,0.076,11,34,0.99780,3.51,0.56,9.4,5
11,7.5,0.500,0.36,6.10,0.071,17,102,0.99780,3.35,0.80,10.5,5
27,7.9,0.430,0.21,1.60,0.106,10,37,0.99660,3.17,0.91,9.5,5
40,7.3,0.450,0.36,5.90,0.074,12,87,0.99780,3.33,0.83,10.5,5
65,7.2,0.725,0.05,4.65,0.086,4,11,0.99620,3.41,0.39,10.9,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1563,7.2,0.695,0.13,2.00,0.076,12,20,0.99546,3.29,0.54,10.1,5
1564,7.2,0.695,0.13,2.00,0.076,12,20,0.99546,3.29,0.54,10.1,5
1567,7.2,0.695,0.13,2.00,0.076,12,20,0.99546,3.29,0.54,10.1,5
1581,6.2,0.560,0.09,1.70,0.053,24,32,0.99402,3.54,0.60,11.3,5


In [11]:
wine_data.drop_duplicates(inplace=True)

Избавление от дубликатов улучшает метрику accuracy на тестовой выборке. И снижает разброс - variance.





Проверка наличия пропусков

In [12]:
wine_data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

Для начала подготовим данные к бинарной классификации.

Условно разделим вино на хорошее и плохое. 

Хорошим вином будем называть вино, параметр quality которого  6, плохим — всё остальное.

In [13]:
wine_data['quality'] = wine_data['quality'].apply(lambda x: 1 if x >= 6 else 0)

In [14]:
TARGET_FEATURE = 'quality'

In [15]:
X = wine_data.drop(TARGET_FEATURE, axis=1)
y = wine_data[TARGET_FEATURE]

In [16]:
y.value_counts(True)

1    0.529065
0    0.470935
Name: quality, dtype: float64

Выборка достаточно равномерно разделена по классам. Можно использовать метрику accuracy,

но в методе сплитования все же лучше использовать stratify=y

Затем разделим выборку на тренировочную и тестовую:

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
# Создаем объект класса случайный лес
rf_clf_full = ensemble.RandomForestClassifier(
    n_estimators=500, # число деревьев
    criterion='gini', # критерий эффективности
    max_depth=8, # максимальная глубина дерева
    min_samples_leaf=10, # минимальное число объектов в листе
    max_features='sqrt', # число признаков из метода случайных подространств
    random_state=42 # генератор случайных чисел
)

# Обучаем модель 
rf_clf_full.fit(X_train, y_train)

RandomForestClassifier(max_depth=8, min_samples_leaf=10, n_estimators=500,
                       random_state=42)

In [19]:
# Делаем предсказание класса
y_train_pred = rf_clf_full.predict(X_train)

# Выводим отчет о метриках
print(metrics.classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.84      0.81      0.82       505
           1       0.84      0.86      0.85       582

    accuracy                           0.84      1087
   macro avg       0.84      0.84      0.84      1087
weighted avg       0.84      0.84      0.84      1087



In [20]:
# Делаем предсказание класса
y_test_pred = rf_clf_full.predict(X_test)

# Выводим отчет о метриках
print(metrics.classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.80      0.73      0.76       135
           1       0.76      0.82      0.79       137

    accuracy                           0.78       272
   macro avg       0.78      0.78      0.78       272
weighted avg       0.78      0.78      0.78       272

